# Using AI21 Summarize on SageMaker through Model Packages

This sample notebook shows you how to deploy **AI21 Summarize** using Amazon SageMaker.


--------------------
## <font color='orange'>Important:</font>
Please visit model detail page in <a href="https://aws.amazon.com/marketplace/pp/prodview-dkwy6chb63hk2">https://aws.amazon.com/marketplace/pp/prodview-dkwy6chb63hk2</a> to learn more. <font color='orange'>If you do not have access to the link, please contact account admin for the help.</font>

You will find details about the model including pricing, supported region, and end user license agreement. To use the model, please click “<font color='orange'>Continue to Subscribe</font>” from the detail page, come back here and learn how to deploy and inference.


-------------------

**AI21 Summarize** model offers access to our top-of-the-line summarization engine. It has been specifically developed for understanding long texts and providing a faithful summary of the original document. As a task-specific model, it can be easily integrated into existing systems without requiring prompt engineering: simply provide the text you want to summarize and get the best summary in the market (according to a [recent study](https://www.ai21.com/blog/summarize-api-outperforms-openais-models)).



## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Summarize a single paragraph](#B.-Summarize-a-single-paragraph)
   1. [Summarize longer texts](#C.-Summarize-longer-texts)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## Imports

In [16]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

In [18]:
# Import wikipedia for longer texts example
! pip install wikipedia
import wikipedia


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [5]:
boto3.__version__

'1.33.9'

### Install ai21 python SDK

In [19]:
! pip install -U "ai21[AWS]>=2.0.3"
import ai21


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Check the version of ai21 - must be 2.0.3 or higher

In [7]:
import ai21
ai21.__version__

'2.1.3'

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [8]:
region = boto3.Session().region_name

# Get the updated ARN
model_package_arn = ai21.SageMaker.get_model_package_arn(model_name="summarize", region=region)

In [9]:
role = get_execution_role()
sagemaker_session = sage.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## 2. Create an endpoint and perform real-time inference

### <span style='color:Blue'> How to choose the best instance for my use case?</span>
<span style='color:#0057FF'> When you create your endpoint, you need to choose the instance type to run the model on. Choosing the right instance is mainly a matter of economics. Depending on your use case, you probably want the most cost-effective instance possible. In this notebook we use one of the supported instances.</span>

<span style='color:#0057FF'>Looking for the list of all supported instances? See</span> [here](https://docs.ai21.com/docs/choosing-the-right-instance-type-for-amazon-sagemaker-models#ai21-summarize).

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [10]:
endpoint_name = "summarize"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.12xlarge"
)

### A. Create an endpoint

In [11]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name, 
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=600,
)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Summarize a single paragraph

**AI21 Summarize model** offers access to our world-class summarization engine. It has been specifically developed for understanding long texts and providing a faithful summary of the original document.

This model takes a piece of text and generates summaries that remain faithful to the original document (i.e. no external information is added during the process).

The input text should contain **at least 40 words** and **no more than 50,000 characters**.

### Important - Model Usage
The input to the model should be simple and straight-forward, no prompting needed.

**Good input:** *[TEXT]*

**Bad input:** *Summarize the following text: [TEXT]*

First, create an AI21 Sagemaker client

In [12]:
from ai21 import AI21SageMakerClient
from ai21.models import DocumentType

client = AI21SageMakerClient(endpoint_name=endpoint_name)

The following is an example of a summary of a single paragraph taken from a news article.

In [22]:
text = """The error affected a number of international flights leaving the terminal on Wednesday, with some airlines urging passengers to travel only with hand luggage.
Virgin Atlantic said all airlines flying out of the terminal had been affected.
Passengers have been warned it may be days before they are reunited with luggage.
An airport spokesperson apologised and said the fault had now been fixed.
Virgin Atlantic said it would ensure all bags were sent out as soon as possible.
It added customers should retain receipts for anything they had bought and make a claim to be reimbursed.
Passengers, who were informed by e-mail of the problem, took to social media to vent their frustrations.
One branded the situation "ludicrous" and said he was only told 12 hours before his flight.
The airport said it could not confirm what the problem was, what had caused it or how many people had been affected."""

response = client.summarize.create(
    source=text,
    source_type=DocumentType.TEXT,
)

print("Original text:")
print(text)
print("================")
print("Summary:")
print(response.summary)

Original text:
The error affected a number of international flights leaving the terminal on Wednesday, with some airlines urging passengers to travel only with hand luggage.
Virgin Atlantic said all airlines flying out of the terminal had been affected.
Passengers have been warned it may be days before they are reunited with luggage.
An airport spokesperson apologised and said the fault had now been fixed.
Virgin Atlantic said it would ensure all bags were sent out as soon as possible.
It added customers should retain receipts for anything they had bought and make a claim to be reimbursed.
Passengers, who were informed by e-mail of the problem, took to social media to vent their frustrations.
One branded the situation "ludicrous" and said he was only told 12 hours before his flight.
The airport said it could not confirm what the problem was, what had caused it or how many people had been affected.
Summary:
A number of international flights left the terminal on Wednesday with baggage pr

### C. Summarize longer texts

AI21 Summarize can handle inputs up to 50,000 characters, as stated above. This translates into roughly 10,000 words, or a whopping 40 pages.

As a demonstration of the model's behavior, we'll load a page from Wikipedia. If you're a fan of the TV show 'The Office', this will look familiar :)

In [23]:
wiki_page = wikipedia.page("The_Office_(American_TV_series)").content

Since this is a full page, we will focus only the opening section.

In [24]:
office_text = wiki_page.split("\n\n")[0]
print(office_text)

The Office is an American mockumentary sitcom television series that depicts the everyday work lives of office employees at the Scranton, Pennsylvania, branch of the fictional Dunder Mifflin Paper Company. It aired on NBC from March 24, 2005, to May 16, 2013, with a total of nine seasons consisting of 201 episodes. Based on the 2001–2003 BBC series of the same name created by (and starring) Ricky Gervais and Stephen Merchant, the show was created by Greg Daniels, a veteran writer for Saturday Night Live, King of the Hill, and The Simpsons. It was co-produced by Daniels' Deedle-Dee Productions and Reveille Productions (later Shine America), in association with Universal Television. The original executive producers were Daniels, Gervais, Merchant, Howard Klein and Ben Silverman, with numerous others being promoted in later seasons.
Like its British counterpart, the series was filmed in a single-camera setup without a studio audience or a laugh track, to mirror the look of an actual docum

The summaries are formatted as bullet lists, following the original text flow.

In [26]:
response = client.summarize.create(
    source=office_text,
    source_type=DocumentType.TEXT,
)

print(response.summary)

The Office is a mockumentary sitcom television series about the everyday work lives of office employees.
Based on the BBC series of the same name, the show was created by Greg Daniels and co-produced by Deedle-Dee Productions and Reveille Productions (later Shine America). It aired on NBC from 2001 to 2003 and featured Steve Carell, Rainn Wilson, John Krasinski, Jenna Fischer and B. J. Novak.
The Office received moderately positive reviews from critics during its short first season, but received significant acclaim from television critics in the following seasons, winning several awards. The eighth season was criticized for a decline in quality, but the ninth season ended the series with a generally positive response.


Here is the full text and the bullet points summarized:

In [ ]:
print("Original text:")
print(office_text)
print("====================")
print("Summary:")
summaries = response.summary.split("\n")
for summary in summaries:
    print("- " + summary)

### Interested in learning more?
Take a look at our [guide](https://docs.ai21.com/docs/summarize-api) to understand all the capabilities of AI21 Summarize model

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()